In [1]:
# Import all required libraries for Phase 2
import os
import re
import json
import logging
import pandas as pd
from datetime import datetime
from typing import List, Dict, Optional, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

# PDF processing libraries
try:
    import pikepdf
    print(" pikepdf imported successfully")
except ImportError:
    print(" pikepdf not found. Run: pip install pikepdf")

try:
    import PyPDF2
    print(" PyPDF2 imported successfully")
except ImportError:
    print(" PyPDF2 not found. Run: pip install PyPDF2")

try:
    import pdfplumber
    print(" pdfplumber imported successfully")
except ImportError:
    print(" pdfplumber not found. Run: pip install pdfplumber")

# Progress tracking
from tqdm import tqdm
import time

print(f" Phase 2 started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(" Processing 195 PDF files with password-protected access")
print("=" * 60)


 pikepdf imported successfully
 PyPDF2 imported successfully
 pdfplumber imported successfully
 Phase 2 started at: 2025-12-18 22:50:48
 Processing 195 PDF files with password-protected access


In [ ]:
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # Fall back to env vars already set in shell

# Configuration for processing your PDF files
CONFIG = {
    "input_folder": os.environ.get("DOWNLOAD_FOLDER", "data/raw_emails"),
    "output_folder": os.environ.get("OUTPUT_FOLDER", "data/raw_emails/output"),
    "excel_filename": "credit_card_statements_195_files.xlsx",

    # Passwords loaded from environment variables — never hardcode these
    "passwords": {
        "HDFC": os.environ.get("HDFC_PDF_PASSWORD", ""),
        "IDFC": os.environ.get("IDFC_PDF_PASSWORD", "")
    },

    # Backup passwords: add variations in your .env if needed
    "backup_passwords": {
        "HDFC": [p for p in [os.environ.get("HDFC_PDF_PASSWORD", "")] if p],
        "IDFC": [p for p in [os.environ.get("IDFC_PDF_PASSWORD", "")] if p]
    },

    # Processing settings
    "batch_size": 20,
    "max_retries": 3
}

# Validate that passwords are loaded
if not CONFIG["passwords"]["HDFC"] or not CONFIG["passwords"]["IDFC"]:
    print("⚠️  WARNING: One or more PDF passwords are not set.")
    print("    Copy .env.example → .env and set HDFC_PDF_PASSWORD and IDFC_PDF_PASSWORD.")
else:
    print("✅ PDF passwords loaded from environment variables.")

# Create output directory
os.makedirs(CONFIG["output_folder"], exist_ok=True)

# Setup comprehensive logging for error tracking
import logging
log_file = os.path.join(CONFIG["output_folder"], "processing_log.txt")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file, 'w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print("📁 Configuration loaded successfully:")
print(f"   Input Folder:  {CONFIG['input_folder']}")
print(f"   Output Folder: {CONFIG['output_folder']}")
print(f"   Excel Output:  {CONFIG['excel_filename']}")
print(f"   HDFC Password: {'[SET]' if CONFIG['passwords']['HDFC'] else '[NOT SET]'}")
print(f"   IDFC Password: {'[SET]' if CONFIG['passwords']['IDFC'] else '[NOT SET]'}")
print(f"   Log File:      {log_file}")

In [5]:
# Analyze all PDF files in your folder
input_folder = CONFIG["input_folder"]

if not os.path.exists(input_folder):
    print(f" Input folder not found: {input_folder}")
    print(" Make sure Phase 1 was completed and PDFs are downloaded")
else:
    # Get all PDF files
    all_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.pdf')]
    
    # Categorize by bank based on filename (as you specified)
    hdfc_files = [f for f in all_files if f.upper().startswith('HDFC')]
    idfc_files = [f for f in all_files if f.upper().startswith('IDFC')]
    other_files = [f for f in all_files if not f.upper().startswith(('HDFC', 'IDFC'))]
    
    print(f" FILE ANALYSIS:")
    print(f"   Total PDF files found: {len(all_files)}")
    print(f"   HDFC files: {len(hdfc_files)}")
    print(f"   IDFC files: {len(idfc_files)}")
    print(f"   Other/Unknown files: {len(other_files)}")
    
    # Verify count
    if len(all_files) == 195:
        print(" Perfect! All 195 files confirmed")
    else:
        print(f"  Expected 195 files, found {len(all_files)}")
    
    # Show sample filenames for verification
    print(f"\n SAMPLE FILENAMES:")
    if hdfc_files:
        print(f"   HDFC Example: {hdfc_files[0]}")
    if idfc_files:
        print(f"   IDFC Example: {idfc_files}")
    if other_files:
        print(f"   Other Example: {other_files}")


 FILE ANALYSIS:
   Total PDF files found: 187
   HDFC files: 99
   IDFC files: 88
   Other/Unknown files: 0
  Expected 195 files, found 187

 SAMPLE FILENAMES:
   HDFC Example: HDFC_20250812_181915_jumbo_loan_0001019020020455148.pdf
   IDFC Example: ['IDFC_20250812_181912_20000002170120_22072025_111137192.pdf', 'IDFC_20250812_181913_574101XXXX.pdf', 'IDFC_20250812_181914_574101XXXX.pdf', 'IDFC_20250812_181916_90300001872873_20072025_120319539.pdf', 'IDFC_20250812_181917_574101XXXX.pdf', 'IDFC_20250812_181918_90300001872873_20052025_133715846.pdf', 'IDFC_20250812_181919_20000002170120_23042025_115543174.pdf', 'IDFC_20250812_181920_20000002170120_22022025_143507512.pdf', 'IDFC_20250812_181923_574101XXXX.pdf', 'IDFC_20250812_181924_20000002170120_22052025_120616270.pdf', 'IDFC_20250812_181924_574101XXXX.pdf', 'IDFC_20250812_181925_574101XXXX.pdf', 'IDFC_20250812_181926_90300001872873_20062025_131901330.pdf', 'IDFC_20250812_181927_20000002170120_22062025_115751719.pdf', 'IDFC_20250812_1819

In [6]:
class CreditCardPDFProcessor:
    """
    Robust PDF processor for 195 files with comprehensive error handling
    Handles password-protected PDFs from HDFC and IDFC banks
    """
    
    def __init__(self, config):
        self.config = config
        self.processed_records = []
        self.failed_files = []
        self.stats = {
            'total_processed': 0,
            'successful': 0,
            'failed': 0,
            'hdfc_success': 0,
            'idfc_success': 0,
            'password_failures': 0,
            'text_extraction_failures': 0,
            'amount_extraction_failures': 0
        }
    
    def identify_bank_from_filename(self, filename):
        """Identify bank from filename as you specified"""
        filename_upper = filename.upper()
        if filename_upper.startswith('HDFC'):
            return 'HDFC'
        elif filename_upper.startswith('IDFC'):
            return 'IDFC'
        else:
            return 'UNKNOWN'
    
    def try_pdf_passwords(self, pdf_path, bank):
        """Try multiple passwords to open PDF"""
        # Start with your specific password, then try backups
        passwords_to_try = [self.config['passwords'][bank]] + self.config['backup_passwords'][bank]
        
        for password in passwords_to_try:
            try:
                pdf = pikepdf.Pdf.open(pdf_path, password=password)
                logger.info(f" PDF opened: {os.path.basename(pdf_path)} with password")
                return pdf, password
            except Exception as e:
                continue
        
        self.stats['password_failures'] += 1
        logger.error(f" All passwords failed for: {os.path.basename(pdf_path)}")
        return None, None
    
    def extract_text_with_multiple_methods(self, pdf_path, bank, password):
        """Extract text using multiple fallback methods"""
        text = ""
        
        # Method 1: pdfplumber (best for formatted text)
        try:
            with pdfplumber.open(pdf_path, password=password) as pdf:
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                if text.strip():
                    return text
        except Exception as e:
            logger.debug(f"pdfplumber failed for {os.path.basename(pdf_path)}: {e}")
        
        # Method 2: PyPDF2 (good backup)
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                if pdf_reader.is_encrypted:
                    success = pdf_reader.decrypt(password)
                    if not success:
                        raise Exception("PyPDF2 decryption failed")
                
                for page in pdf_reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                if text.strip():
                    return text
        except Exception as e:
            logger.debug(f"PyPDF2 failed for {os.path.basename(pdf_path)}: {e}")
        
        self.stats['text_extraction_failures'] += 1
        return ""
    
    def extract_amount_from_text(self, text, bank):
        """Extract bill amount using bank-specific patterns"""
        # Bank-specific amount extraction patterns
        amount_patterns = {
            'HDFC': [
                r'Total Amount Due\s*[:\-]?\s*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
                r'Amount Due\s*[:\-]?\s*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
                r'Outstanding Balance\s*[:\-]?\s*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
                r'Current Balance\s*[:\-]?\s*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)'
            ],
            'IDFC': [
                r'Total Amount Due\s*[:\-]?\s*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
                r'Amount Payable\s*[:\-]?\s*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
                r'Current Balance\s*[:\-]?\s*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
                r'Outstanding Amount\s*[:\-]?\s*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)'
            ]
        }
        
        for pattern in amount_patterns.get(bank, []):
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                try:
                    # Clean the amount string and convert to float
                    amount_str = matches[0].replace(',', '').strip()
                    amount = float(amount_str)
                    if amount > 0:  # Valid positive amount
                        logger.info(f" Amount found: ₹{amount:,.2f} for {bank}")
                        return amount
                except ValueError:
                    continue
        
        self.stats['amount_extraction_failures'] += 1
        return None
    
    def extract_dates_from_text(self, text, bank):
        """Extract statement and due dates"""
        date_patterns = {
            'HDFC': [
                r'Statement Date\s*[:\-]?\s*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
                r'Payment Due Date\s*[:\-]?\s*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
                r'Bill Date\s*[:\-]?\s*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})'
            ],
            'IDFC': [
                r'Statement Date\s*[:\-]?\s*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
                r'Due Date\s*[:\-]?\s*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
                r'Payment Due Date\s*[:\-]?\s*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})'
            ]
        }
        
        dates = {}
        for pattern in date_patterns.get(bank, []):
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                if 'Statement' in pattern or 'Bill' in pattern:
                    dates['statement_date'] = matches[0]
                elif 'Due' in pattern or 'due' in pattern:
                    dates['due_date'] = matches
        
        return dates
    
    def standardize_date_format(self, date_str):
        """Convert date to your required format and extract components"""
        if not date_str:
            return "", "", ""
        
        # Try different date formats
        date_formats = ['%d/%m/%Y', '%d-%m-%Y', '%Y/%m/%d', '%Y-%m-%d']
        
        for fmt in date_formats:
            try:
                date_obj = datetime.strptime(date_str, fmt)
                return (
                    date_obj.strftime('%d/%m/%Y'),  # DATE column
                    date_obj.strftime('%B'),        # MONTH column
                    str(date_obj.year)              # YEAR column
                )
            except ValueError:
                continue
        
        # Manual parsing for DD/MM/YYYY or DD-MM-YYYY format
        if re.match(r'\d{1,2}[/-]\d{1,2}[/-]\d{4}', date_str):
            try:
                parts = re.split('[/-]', date_str)
                day, month, year = parts[0], parts[1], parts[2]
                date_obj = datetime(int(year), int(month), int(day))
                return (
                    f"{day.zfill(2)}/{month.zfill(2)}/{year}",
                    date_obj.strftime('%B'),
                    year
                )
            except:
                pass
        
        return date_str, "", ""
    
    def process_single_pdf_file(self, pdf_filename):
        """Process a single PDF file with comprehensive error handling"""
        pdf_path = os.path.join(self.config['input_folder'], pdf_filename)
        
        try:
            # Step 1: Identify bank from filename
            bank = self.identify_bank_from_filename(pdf_filename)
            if bank == 'UNKNOWN':
                raise Exception(f"Cannot identify bank from filename: {pdf_filename}")
            
            # Step 2: Open PDF with password
            pdf_obj, password_used = self.try_pdf_passwords(pdf_path, bank)
            if not pdf_obj:
                raise Exception(f"Failed to open PDF with any password: {pdf_filename}")
            
            # Close the PDF object immediately to free memory
            pdf_obj.close()
            
            # Step 3: Extract text from PDF
            text_content = self.extract_text_with_multiple_methods(pdf_path, bank, password_used)
            if not text_content:
                raise Exception(f"No text extracted from PDF: {pdf_filename}")
            
            # Step 4: Extract amount (your main requirement)
            amount = self.extract_amount_from_text(text_content, bank)
            if amount is None:
                raise Exception(f"No amount found in PDF: {pdf_filename}")
            
            # Step 5: Extract dates
            dates = self.extract_dates_from_text(text_content, bank)
            statement_date = dates.get('statement_date', '')
            
            # If no date found in PDF, try to extract from filename or use current date
            if not statement_date:
                date_match = re.search(r'([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})', pdf_filename)
                if date_match:
                    statement_date = date_match.group(1)
                else:
                    statement_date = datetime.now().strftime('%d/%m/%Y')
            
            # Step 6: Standardize date format for your columns
            date_formatted, month_name, year = self.standardize_date_format(statement_date)
            
            # Step 7: Create record with your exact column requirements
            record = {
                'DATE': date_formatted,           # Your required column
                'MONTH': month_name,              # Your required column  
                'YEAR': year,                     # Your required column
                'BANK': bank,                     # Your required column
                'AMOUNT': amount,                 # Your required column
                'STATEMENT_DATE': statement_date,  # Additional info
                'DUE_DATE': dates.get('due_date', ''),  # Additional info
                'FILE_NAME': pdf_filename,        # For tracking
                'PROCESSED_TIME': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            
            # Update success statistics
            self.stats['successful'] += 1
            if bank == 'HDFC':
                self.stats['hdfc_success'] += 1
            elif bank == 'IDFC':
                self.stats['idfc_success'] += 1
            
            logger.info(f" SUCCESS: {bank} - ₹{amount:,.2f} - {month_name} {year}")
            return record
            
        except Exception as error:
            # Record the failure with details
            failure_info = {
                'filename': pdf_filename,
                'error_message': str(error),
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'bank': self.identify_bank_from_filename(pdf_filename)
            }
            self.failed_files.append(failure_info)
            self.stats['failed'] += 1
            
            logger.error(f" FAILED: {pdf_filename} - {error}")
            return None
        
        finally:
            self.stats['total_processed'] += 1
    
    def process_all_195_files(self, file_list):
        """Process all PDF files with batch processing and progress tracking"""
        logger.info(f" Starting processing of {len(file_list)} PDF files")
        
        # Process files in batches to manage memory
        batch_size = self.config['batch_size']
        
        for i in range(0, len(file_list), batch_size):
            batch_files = file_list[i:i + batch_size]
            batch_number = i // batch_size + 1
            total_batches = (len(file_list) + batch_size - 1) // batch_size
            
            print(f"\n Processing Batch {batch_number}/{total_batches}: {len(batch_files)} files")
            
            # Process each file in the batch with progress bar
            batch_successful = 0
            for pdf_file in tqdm(batch_files, desc=f"Batch {batch_number}", leave=False):
                record = self.process_single_pdf_file(pdf_file)
                if record:
                    self.processed_records.append(record)
                    batch_successful += 1
                
                # Small delay to prevent system overload
                time.sleep(0.05)
            
            # Batch summary
            print(f"    Batch {batch_number} completed: {batch_successful}/{len(batch_files)} successful")
            print(f"    Overall progress: {len(self.processed_records)} successful, {self.stats['failed']} failed")
            
            # Memory cleanup suggestion
            if batch_number % 5 == 0:  # Every 5 batches
                print(f"  Processed {batch_number * batch_size} files. Consider memory cleanup if needed.")
        
        logger.info(f" All files processed: {self.stats['successful']} successful, {self.stats['failed']} failed")
        return self.processed_records

# Initialize the processor
processor = CreditCardPDFProcessor(CONFIG)
print(" PDF Processor initialized successfully!")


 PDF Processor initialized successfully!


In [7]:
# Start processing all your PDF files
print(" STARTING PHASE 2: PROCESSING 195 PDF FILES")
print(f" Started at: {datetime.now().strftime('%H:%M:%S')}")
print("=" * 60)

# Process all files
processed_records = processor.process_all_195_files(all_files)

print("\n PROCESSING COMPLETED!")
print(f" Finished at: {datetime.now().strftime('%H:%M:%S')}")
print("=" * 60)

# Display processing statistics
print(f" PROCESSING SUMMARY:")
print(f"   Total files processed: {processor.stats['total_processed']}")
print(f"   Successful extractions: {processor.stats['successful']}")
print(f"   Failed extractions: {processor.stats['failed']}")
print(f"   Success rate: {(processor.stats['successful']/len(all_files)*100):.1f}%")
print(f"   HDFC successful: {processor.stats['hdfc_success']}")
print(f"   IDFC successful: {processor.stats['idfc_success']}")
print(f"   Password failures: {processor.stats['password_failures']}")
print(f"   Text extraction failures: {processor.stats['text_extraction_failures']}")
print(f"   Amount extraction failures: {processor.stats['amount_extraction_failures']}")


2025-12-18 22:53:37,090 - INFO -  Starting processing of 187 PDF files


 STARTING PHASE 2: PROCESSING 195 PDF FILES
 Started at: 22:53:37

 Processing Batch 1/10: 20 files


Batch 1:   0%|          | 0/20 [00:00<?, ?it/s]2025-12-18 22:53:38,367 - INFO -  PDF opened: HDFC_20250812_181915_jumbo_loan_0001019020020455148.pdf with password
2025-12-18 22:53:39,378 - ERROR -  FAILED: HDFC_20250812_181915_jumbo_loan_0001019020020455148.pdf - No amount found in PDF: HDFC_20250812_181915_jumbo_loan_0001019020020455148.pdf
Batch 1:   5%|▌         | 1/20 [00:01<00:37,  1.98s/it]2025-12-18 22:53:39,525 - INFO -  PDF opened: HDFC_20250812_181921_jumbo_loan_0001019020020455148.pdf with password
2025-12-18 22:53:40,231 - ERROR -  FAILED: HDFC_20250812_181921_jumbo_loan_0001019020020455148.pdf - No amount found in PDF: HDFC_20250812_181921_jumbo_loan_0001019020020455148.pdf
Batch 1:  10%|█         | 2/20 [00:02<00:23,  1.32s/it]2025-12-18 22:53:40,366 - INFO -  PDF opened: HDFC_20250812_181922_jumbo_loan_0001019020020455148.pdf with password
2025-12-18 22:53:41,058 - ERROR -  FAILED: HDFC_20250812_181922_jumbo_loan_0001019020020455148.pdf - No amount found in PDF: HDFC_202

    Batch 1 completed: 11/20 successful
    Overall progress: 11 successful, 9 failed

 Processing Batch 2/10: 20 files


Batch 2:   0%|          | 0/20 [00:00<?, ?it/s]2025-12-18 22:54:01,324 - INFO -  PDF opened: HDFC_20250812_182012_5241XXXXXXXXXX70_15-04-2024.PDF with password
--- Logging error ---
Traceback (most recent call last):
  File "D:\anaconda3\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "D:\anaconda3\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\u20b9' in position 48: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "e:\Power BI Project\My Own Data\Credit Card Emi\credit_card_env\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "e:\Power BI Project\My Own Data\Credit Card Emi\credit_card_env\Lib\si

...

    Batch 8 completed: 0/20 successful
    Overall progress: 50 successful, 110 failed

 Processing Batch 9/10: 20 files


Batch 9:   0%|          | 0/20 [00:00<?, ?it/s]2025-12-18 22:56:19,035 - ERROR -  All passwords failed for: IDFC_20250812_182101_574101XXXX.pdf
2025-12-18 22:56:19,036 - ERROR -  FAILED: IDFC_20250812_182101_574101XXXX.pdf - Failed to open PDF with any password: IDFC_20250812_182101_574101XXXX.pdf
Batch 9:   5%|▌         | 1/20 [00:00<00:02,  8.94it/s]2025-12-18 22:56:19,357 - INFO -  PDF opened: IDFC_20250812_182102_20000002170120_22062023_121719205.pdf with password
2025-12-18 22:56:21,073 - ERROR -  FAILED: IDFC_20250812_182102_20000002170120_22062023_121719205.pdf - No amount found in PDF: IDFC_20250812_182102_20000002170120_22062023_121719205.pdf
Batch 9:  10%|█         | 2/20 [00:02<00:22,  1.25s/it]2025-12-18 22:56:21,323 - INFO -  PDF opened: IDFC_20250812_182105_20000002170120_22092023_125719301.pdf with password
2025-12-18 22:56:23,360 - ERROR -  FAILED: IDFC_20250812_182105_20000002170120_22092023_125719301.pdf - No amount found in PDF: IDFC_20250812_182105_20000002170120_22

    Batch 9 completed: 0/20 successful
    Overall progress: 50 successful, 130 failed

 Processing Batch 10/10: 7 files


Batch 10:   0%|          | 0/7 [00:00<?, ?it/s]2025-12-18 22:56:34,945 - INFO -  PDF opened: IDFC_20250812_182151_20000002170120_22062022_160943149.pdf with password
2025-12-18 22:56:35,818 - ERROR -  FAILED: IDFC_20250812_182151_20000002170120_22062022_160943149.pdf - No amount found in PDF: IDFC_20250812_182151_20000002170120_22062022_160943149.pdf
Batch 10:  14%|█▍        | 1/7 [00:01<00:06,  1.01s/it]2025-12-18 22:56:35,916 - INFO -  PDF opened: IDFC_20250812_182153_20000002170120_22072022_141048333.pdf with password
2025-12-18 22:56:37,218 - ERROR -  FAILED: IDFC_20250812_182153_20000002170120_22072022_141048333.pdf - No amount found in PDF: IDFC_20250812_182153_20000002170120_22072022_141048333.pdf
Batch 10:  29%|██▊       | 2/7 [00:02<00:06,  1.24s/it]2025-12-18 22:56:37,324 - ERROR -  All passwords failed for: IDFC_20250812_182209_574101XXXX.pdf
2025-12-18 22:56:37,326 - ERROR -  FAILED: IDFC_20250812_182209_574101XXXX.pdf - Failed to open PDF with any password: IDFC_20250812_1

    Batch 10 completed: 0/7 successful
    Overall progress: 50 successful, 137 failed
  Processed 200 files. Consider memory cleanup if needed.

 PROCESSING COMPLETED!
 Finished at: 22:56:37
 PROCESSING SUMMARY:
   Total files processed: 187
   Successful extractions: 50
   Failed extractions: 137
   Success rate: 26.7%
   HDFC successful: 50
   IDFC successful: 0
   Password failures: 72
   Text extraction failures: 0
   Amount extraction failures: 65
